In [1]:
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

/usr/local/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [7]:
%%manim -qm PlotNormal

class PlotNormal(ThreeDScene):
    def construct(self):
        resolucao = 24
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Parametrização da normal (f : R^2 \to R)
        def param_gauss(u, v):
            x = u
            y = v
            sigma, mu = 0.4, [0.0, 0.0]
            d = np.linalg.norm(np.array([x - mu[0], y - mu[1]]))
            z = np.exp(-(d ** 2 / (2.0 * sigma ** 2)))
            return np.array([x, y, z])

        # Cria a superfície com a resolução de interpolação dada
        gauss_plane = Surface(
            param_gauss,
            resolution=(resolucao, resolucao),
            v_range=[-2, +2],
            u_range=[-2, +2]
        )

        # Roda a câmera
        self.begin_ambient_camera_rotation(rate=0.4)

        gauss_plane.scale(2, about_point=ORIGIN)
        gauss_plane.set_style(fill_opacity=1, stroke_color=GREEN)
        axes = ThreeDAxes()
        self.add(axes, gauss_plane)
        self.wait(2)

Manim Community v0.19.0

In [4]:
%%manim -qm Lorenz

class Lorenz(ThreeDScene):
    def construct(self):
        self.camera.background_color = DARKER_GRAY
        
        cond_iniciais = [np.array([-1.0, 1.0, 0.0]), 
                         np.array([-0.7, 1.0, 0.0]), 
                         np.array([-1.0, 0.7, 0.0])]
        escala = 0.1

        # A quantidade de segundos que se passa entre cada integração
        epsilon = 0.01

        # Cores de cada PVI
        cores = [RED_B, ORANGE, TEAL]
        curvas = VGroup()

        # para cada combinação de valor inicial e cor, crie uma curva
        for pos_ini, cor in zip(cond_iniciais, cores):
            f = self.trajetoria(pos_ini, dt=epsilon)
            curva = ParametricFunction(
                lambda t: f(t),
                color=cor,
                t_range=np.array([0, 30, epsilon]),  # tempo de simulação é de 0 a 30 segundos, com cálculos a cada epsilon segundos e interpolados entre si
                use_vectorized=False
            )
            curva.scale(escala)

            # Centraliza todas as curvas
            curva.move_to(ORIGIN)
            curvas.add(curva)

        self.set_camera_orientation(phi=80 * DEGREES, theta=45 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.1)

        # equações do sistema de Lorenz. O manim irá, por baixo do tapete, compilar separadamente
        # as letras x, y, z para colori-las separadamente (e por isso o uso do \over ao invés de \frac)
        equacoes = MathTex(
            r"""
            {\mathrm{d} x} \over {\mathrm{d} t} &= \sigma(y - x) \\
            {\mathrm{d} y} \over {\mathrm{d} t} &= x(\rho - z) - y \\
            {\mathrm{d} z} \over {\mathrm{d} t} &= xy - \beta z
            """,
            substrings_to_isolate=["x", "y", "z"]
        ).scale(0.6)
        equacoes.set_color_by_tex('x', RED)
        equacoes.set_color_by_tex('y', GREEN)
        equacoes.set_color_by_tex('z', BLUE)

        # equações no canto superior esquerdo
        equacoes.to_edge(UL)

        # equações não devem se mover junto com a câmera
        self.add_fixed_in_frame_mobjects(equacoes)
        self.play(Write(equacoes), run_time=1.5)

        # crie todas as curvas na lista simultaneamente
        self.play(*[Create(c) for c in curvas], run_time=20, rate_func=linear)

    # Retorna as equações diferenciais do atrator de Lorenz.
    def lorenz_system(self, pos, sigma=10, rho=28, beta=8 / 3):
        x, y, z = pos
        dx_dt = sigma * (y - x)
        dy_dt = x * (rho - z) - y
        dz_dt = x * y - beta * z
        return np.array([dx_dt, dy_dt, dz_dt])

    # Um passo do método de Runge-Kutta de 4ª ordem
    def rk4_step(self, pos, dt):
        k1 = self.lorenz_system(pos)
        k2 = self.lorenz_system(pos + dt * k1 / 2)
        k3 = self.lorenz_system(pos + dt * k2 / 2)
        k4 = self.lorenz_system(pos + dt * k3)
        return pos + dt * (k1 + 2 * k2 + 2 * k3 + k4) / 6

    # Retorna uma função f(t) que dá a posição no tempo t
    # a partir de integração numérica
    def trajetoria(self, start_pos, dt=0.01):
        cache = {0: np.array(start_pos)}
        def f(t):
            # arredonda t para múltiplos de dt
            steps = int(np.round(t / dt))
            if steps in cache:
                return cache[steps]
            # integra até o passo desejado
            pos = cache[max(cache.keys())]
            for i in range(max(cache.keys()) + 1, steps + 1):
                pos = self.rk4_step(pos, dt)
                cache[i] = pos
            return cache[steps]
        return f

Manim Community v0.19.0